# Multi-tenant RAG application using Standard deployment of AI Search

In [20]:
import argparse
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
from collections import Counter
import requests
import uuid
from datetime import datetime
import time
import openai
from azure.identity import DefaultAzureCredential
import dotenv
import logging

dotenv.load_dotenv()

True

In [109]:
EMBEDDING_DIMS=1536
logging.basicConfig(level=logging.INFO)
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT_NAME = (
    os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME") or "embedding"
)
AZURE_SEARCH_SERVICE_ENDPOINT = os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_SERVICE_API_KEY = os.environ.get("AZURE_SEARCH_SERVICE_API_KEY")

AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_CONNECTION_STRING = os.environ.get("AZURE_STORAGE_CONNECTION_STRING")


# Step 0 - Setup the shared Index

In [101]:
text_index_name = "multi-tenant-index-text"
vector_index_name = "multi-tenant-index-vector"
vector_search_profile = "vector-profile-mt"
vector_search_config = "vector-search-config-mt"
vector_search_vectorizer = "vectorizer-mt"
semantic_config_name = "semantic-config-mt"

In [11]:
search_endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
search_api_key = AZURE_SEARCH_SERVICE_API_KEY
search_api_version = "2023-10-01-Preview"
search_headers = {"Content-Type": "application/json", "api-key": search_api_key}

In [108]:
def get_vector_search_config(vector_search_config,vector_search_profile, vector_search_vectorizer, 
                             model_uri, model_name, model_api_key, metric="cosine", m=4, efConstruction=400, efSearch=500):
        """
        Create a vector search configuration
        model_uri: the uri of the embedding model
        model_name: the deployment name of the embedding model
        model_api_key: the api key of the embedding model
        metric: the distance metric to use for the vector search, use cosine for OpenAI models
        m: bi-directional link count
        efConstruction: number of nearest neighbors to consider during indexiing
        efSearch: number of nearest neighbors to consider during search
        """
        config = {
            "algorithms": [
                {
                    "name": vector_search_config,
                    "kind": "hnsw",
                    "hnswParameters": {
                        "metric": metric,
                        "m": m,
                        "efConstruction": efConstruction,  
                        "efSearch": efSearch,  
                    },
                    "exhaustiveKnnParameters": None,
                }
            ],
            "profiles": [
                {
                    "name": vector_search_profile,
                    "algorithm": vector_search_config,
                    "vectorizer": vector_search_vectorizer,
                }
            ],
            "vectorizers": [
                {
                    "name": vector_search_vectorizer,
                    "kind": "azureOpenAI",
                    "azureOpenAIParameters": {
                        "resourceUri": model_uri,
                        "deploymentId": model_name,
                        "apiKey": model_api_key,
                        "authIdentity": None,
                    },
                    "customWebApiParameters": None,
                }
            ],
        }
        return config
    
def get_semantic_config(semantic_config_name):
        config = {
            "defaultConfiguration": None,
            "configurations": [
                {
                    "name": semantic_config_name,
                    "prioritizedFields": {
                        "titleField": None,
                        "prioritizedContentFields": [{"fieldName": "chunk"}],
                        "prioritizedKeywordsFields": [
                            {"fieldName": "id"},
                            {"fieldName": "parent_key"},
                        ],
                    },
                }
            ],
        }
        return config

def check_index_exists(index_name):
        response = requests.get(
            f"{search_endpoint}/indexes('{index_name}')?api-version={search_api_version}",
            headers=search_headers,
        )
        return response.status_code == 200

def delete_index(index_name):
        response = requests.delete(
            f"{search_endpoint}/indexes('{index_name}')?api-version={search_api_version}",
            headers=search_headers,
        )
        logging.info(f"DELETED: {index_name}, {response.status_code}")
        return response.status_code == 204

def get_schema(index_type, vector_search_profile):
        if index_type == "text":
            schema = [
                {
                    "name": "id",
                    "type": "Edm.String",
                    "key": True,
                    "searchable": False,
                    "filterable": True,
                    "sortable": False,
                    "facetable": False,
                },
                {
                    "name": "metadata_storage_path",
                    "type": "Edm.String",
                    "retrievable": True,
                    "searchable": False,
                    "filterable": True,
                    "sortable": False,
                },
                {
                    "name": "content",
                    "type": "Edm.String",
                    "retrievable": True,
                    "searchable": True,
                    "filterable": False,
                    "sortable": False,
                    "facetable": False,
                },
            ]
        elif index_type == "vector":
            schema = [
                {
                    "name": "id",
                    "type": "Edm.String",
                    "key": True,
                    "searchable": True,
                    "filterable": False,
                    "sortable": False,
                    "facetable": False,
                    "analyzer": "keyword",
                },
                {
                    "name": "parent_storage_path",
                    "type": "Edm.String",
                    "key": False,
                    "retrievable": True,
                    "searchable": True,
                    "filterable": False,
                    "sortable": False,
                    "facetable": False,
                },
                {
                    "name": "application_name",
                    "type": "Edm.String",
                    "key": False,
                    "searchable": False,
                    "filterable": True,
                    "sortable": False,
                    "facetable": True,
                },
                {
                    "name": "chunk",
                    "type": "Edm.String",
                    "retrievable": True,
                    "searchable": True,
                    "filterable": False,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "analyzer": "standard.lucene",
                },
                {
                    "name": "parent_key",
                    "type": "Edm.String",
                    "retrievable": True,
                    "searchable": False,
                    "filterable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                },
                {
                    "name": "embedding",
                    "type": "Collection(Edm.Single)",
                    "retrievable": False,
                    "searchable": True,
                    "filterable": False,
                    "sortable": False,
                    "facetable": False,
                    "dimensions": EMBEDDING_DIMS,
                    "vectorSearchProfile": vector_search_profile,
                },
            ]
        
        return schema

def create_index(index_name, schema, scoring_profile = [], vector_search_config=None, semantic_config=None, rebuild = False,):
        if check_index_exists(index_name):
            if rebuild:
                delete_index(index_name)
            else:
                return True
        payload = {
            "name": index_name,
            "defaultScoringProfile": "",
            "fields": schema,
            "scoringProfiles": scoring_profile,
            "similarity": {
                "@odata.type": "#Microsoft.Azure.Search.BM25Similarity",
                "k1": None,
                "b": None,
            },
            "semantic": semantic_config,
            "vectorSearch": vector_search_config,
        }
        response = requests.put(
            f"{search_endpoint}/indexes('{index_name}')?api-version={search_api_version}",
            headers=search_headers,
            json=payload,
        )
        if response.status_code in [200, 201, 204]:
            return True
        else:
            logging.error(f"ERROR: {response.status_code}|| {response.text}")
            return False

In [14]:
vector_search_config = get_vector_search_config(vector_search_config,
                                                vector_search_profile, 
                                                vector_search_vectorizer, 
                                                model_uri=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com", 
                                                model_name=AZURE_OPENAI_DEPLOYMENT_NAME, 
                                                model_api_key=AZURE_OPENAI_API_KEY, 
                                                metric="cosine", m=4, efConstruction=400, efSearch=500)

semantic_config = get_semantic_config(semantic_config_name)
text_index_schema = get_schema(index_type="text", vector_search_profile=None)
vector_index_schema = get_schema(index_type="vector", vector_search_profile=vector_search_profile)

create_index(index_name=text_index_name, 
             schema=text_index_schema, 
             scoring_profile = [], 
             vector_search_config=None, 
             semantic_config=None, 
             rebuild = False,)

create_index(index_name=vector_index_name, 
             schema=vector_index_schema, 
             scoring_profile = [], 
             vector_search_config=vector_search_config, 
             semantic_config=semantic_config, 
             rebuild = False,)


True

# Surface Bot Analyzer

In [17]:
folder_path_t1 ="../data/pdf/"
save_path_t1 = "surface-pro"
app_name_t1 = "surface-pro"

## Step 1 - ingest the data into Blob storage

In [46]:
def upload_files_to_blob(azure_credential, folder_path, save_path="", metadata=None):
    blob_service = BlobServiceClient(
        account_url=f"https://{AZURE_STORAGE_ACCOUNT}.blob.core.windows.net",
        credential=azure_credential,
    )
    blob_container = blob_service.get_container_client(AZURE_STORAGE_CONTAINER)

    if not blob_container.exists():
        logging.info(
            f"Creating blob container {AZURE_STORAGE_CONTAINER} in storage account {AZURE_STORAGE_ACCOUNT}"
        )
        blob_container.create_container()

    logging.info(f"Uploading files to Blob container...")
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            blob_name = f"{save_path}/{file}" if save_path else file
            with open(os.path.join(root, file), "rb") as data:
                blob_container.upload_blob(name=blob_name, data=data, overwrite=True, metadata=metadata)
    logging.info(f"Files uploaded to Blob container {AZURE_STORAGE_CONTAINER}")

In [19]:
azure_credential = DefaultAzureCredential(exclude_shared_token_cache_credential=True)
upload_files_to_blob(azure_credential, folder_path_t1, save_path_t1, metadata={"application_name": app_name_t1})

INFO:azure.identity._credentials.environment:Incomplete environment configuration for EnvironmentCredential. These variables are set: AZURE_TENANT_ID
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.15.0 Python/3.11.8 (Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://stayjdfpgpoyvkc.blob.core.windows.net/text?restype=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.19.

# Step 2 - Create the chunking and Embedding skill

In [24]:
def create_skillset(vector_skillset_name, vector_index_name, model_uri, model_name, model_api_key, chunk_size=2000, overlap_size=500):
    """
    Create a skillset for the indexer
    This skillset will be used to enrich the content before indexing
    """
    skillset_payload = {
        "name": vector_skillset_name,
            "description": "skills required for vector embedding creation processing",
            "skills": [
                {
                    "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
                    "name": "text-chunking-skill",
                    "description": "Skillset to describe the Text chunking required for vectorization",
                    "context": "/document",
                    "defaultLanguageCode": "en",
                    "textSplitMode": "pages",
                    "maximumPageLength": chunk_size,
                    "pageOverlapLength": overlap_size,
                    "maximumPagesToTake": 0,
                    "inputs": [{"name": "text", "source": "/document/content"}],
                    "outputs": [{"name": "textItems", "targetName": "chunks"}],
                },
                {
                    "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
                    "name": "embedding-generation-skill",
                    "description": "",
                    "context": "/document/chunks/*",
                    "resourceUri": model_uri,
                    "apiKey": model_api_key,
                    "deploymentId": model_name,
                    "inputs": [{"name": "text", "source": "/document/chunks/*"}],
                    "outputs": [{"name": "embedding", "targetName": "embedding"}],
                },
            ],
            "indexProjections": {
                "selectors": [
                    {
                        "targetIndexName": vector_index_name,
                        "parentKeyFieldName": "parent_key",
                        "sourceContext": "/document/chunks/*",
                        "mappings": [
                            {
                                "name": "chunk",
                                "source": "/document/chunks/*",
                                "sourceContext": None,
                                "inputs": [],
                            },
                            {
                                "name": "embedding",
                                "source": "/document/chunks/*/embedding",
                                "sourceContext": None,
                                "inputs": [],
                            },
                            {
                                "name": "application_name",
                                "source": "/document/application_name",
                                "sourceContext": None,
                                "inputs": [],
                            },
                            {
                                "name": "parent_storage_path",
                                "source": "/document/metadata_storage_path",
                                "sourceContext": None,
                                "inputs": [],
                            }
                        ],
                    }
                ],
            },
    }

    response = requests.put(
            f"{search_endpoint}/skillsets('{vector_skillset_name}')?api-version={search_api_version}",
            headers=search_headers,
            json=skillset_payload,
        )
    if response.status_code in [200, 201, 204]:
        return True
    else:
        logging.error(f"ERROR: {response.status_code}")
        return False

In [28]:
vector_skillset_name=f"chunking-skillset-{save_path_t1}"
chunking_skill = create_skillset(vector_skillset_name=vector_skillset_name, 
                                 vector_index_name=vector_index_name, 
                                 model_uri=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com", 
                                 model_name=AZURE_OPENAI_DEPLOYMENT_NAME, 
                                 model_api_key=AZURE_OPENAI_API_KEY, )

### Step 3 - Create the Indexer and Run it

In [27]:
def create_data_source_blob_storage(
          data_source_name,
          blob_connection, 
          blob_container_name, 
          query,
    ) -> bool:
        query = "" if query is None else query
        data_source_payload = {
            "name": data_source_name,
            "description": "Data source for Azure Blob storage container",
            "type": "azureblob",
            "credentials": {"connectionString": blob_connection},
            "container": {"name": blob_container_name, "query": query},
            "dataChangeDetectionPolicy": None,
            "dataDeletionDetectionPolicy": None,
        }

        response = requests.put(
            f"{search_endpoint}/datasources('{data_source_name}')?api-version={search_api_version}",
            headers=search_headers,
            json=data_source_payload,
        )
        if response.status_code in [200, 201, 204]:
            # self.data_source = response.json()
            return True
        else:
            logging.error(f"ERROR: {response.json()}")
            logging.error(f"ERROR: {response.status_code}")
            return False

def create_indexer(
          indexer_name, 
          search_index_name,
          data_source_name,
          vector_skillset_name,
          cache_storage_connection, 
          parsing_mode="default", 
          disable_at_creation=False, 
          batch_size=1,
          max_failed_items=100,
          output_field_mapping=[],
          ):
        """
        Create an indexer to index the data source
        cache_storage_connection: connection string to the storage account for caching
        parsing_mode: the mode to use for parsing the data source, "text", "delimitedText","json","jsonArray","jsonLines"
        """
        if check_index_exists(search_index_name):
            indexer_payload = {
                "name": indexer_name,
                "description": "Indexer for Azure Blob storage container",
                "dataSourceName": data_source_name,
                "targetIndexName": search_index_name,
                "skillsetName": vector_skillset_name,
                "disabled" : disable_at_creation,
                "parameters": {
                    "configuration": {
                        "parsingMode": parsing_mode,
                        "dataToExtract": "contentAndMetadata",
                    },
                    "batchSize": batch_size,
                    "maxFailedItems": max_failed_items,
                },
                "outputFieldMappings": output_field_mapping,
                "cache": {
                    "enableReprocessing": True,
                    "storageConnectionString": cache_storage_connection,
                },
            }
            response = requests.put(
                f"{search_endpoint}/indexers('{indexer_name}')?api-version={search_api_version}",
                headers=search_headers,
                json=indexer_payload,
            )
            if response.status_code in [200, 201, 204]:
                # self.indexer = response.json()
                return True
            else:
                logging.error(f"ERROR: {response.status_code}, {response.text}")
                return False
        else:
            return False

In [29]:
data_source_name=f"data-source-{save_path_t1}"
indexer_name = f"multi-tenant-indexer-{save_path_t1}"
create_data_source_blob_storage(
          data_source_name=data_source_name,
          blob_connection=AZURE_STORAGE_CONNECTION_STRING, 
          blob_container_name=AZURE_STORAGE_CONTAINER, 
          query=save_path_t1
)
create_indexer(indexer_name = indexer_name, 
          search_index_name=text_index_name,
          data_source_name=data_source_name,
          vector_skillset_name=vector_skillset_name,
          cache_storage_connection=AZURE_STORAGE_CONNECTION_STRING,
)

True

### Step 4 - Run query against the index 

In [99]:
def query_ai_search(query, search_type, search_index_name, text_fields, top=10, semantic_config_name="", select_fields=None, vector_fields=None, application_name = None):
        """
        Query the search index
        """
        search_payload = {
            "top": top,
        }
        if search_type in ["text","hybrid"]:
            search_payload["search"] = query
            search_payload["searchFields"] = ",".join(text_fields)
            search_payload["queryType"] = "simple"
            if len(semantic_config_name) > 0:
                search_payload["queryType"] = "semantic"
                search_payload["semanticConfiguration"] = semantic_config_name                
        if application_name:
            search_payload["filter"] = f"application_name eq '{application_name}'"
        if select_fields:
            search_payload["select"] = ",".join(select_fields)
        if vector_fields and search_type in ["vector","hybrid"]:
            search_payload["vectorQueries"] = [
                {
                    "fields": ",".join(vector_fields),
                    "text": query,
                    "k": top,
                    "kind": "text"
                }
            ]
            search_payload["vectorFilterMode"] = "preFilter"
        logging.info(search_payload)
        response = requests.post(
            f"{search_endpoint}/indexes('{search_index_name}')/docs/search?api-version={search_api_version}",
            headers=search_headers,
            json=search_payload,
        )
        if response.status_code == 200:
            return response.json()
        else:
            logging.error(f"ERROR: {response.status_code}")
            return None

In [105]:
QUERY = "Microsoft Surface Pro"
query_results = query_ai_search(query=QUERY,
                                search_type="hybrid", 
                search_index_name=vector_index_name, 
                text_fields=["chunk"], 
                top=10,  
                #semantic_config_name=semantic_config_name,
                select_fields=["parent_storage_path", "application_name", "chunk"], 
                vector_fields=["embedding"], 
                application_name = app_name_t1)

INFO:root:{'top': 10, 'search': 'Microsoft Surface Pro', 'searchFields': 'chunk', 'queryType': 'simple', 'filter': "application_name eq 'surface-pro'", 'select': 'parent_storage_path,application_name,chunk', 'vectorQueries': [{'fields': 'embedding', 'text': 'Microsoft Surface Pro', 'k': 10, 'kind': 'text'}], 'vectorFilterMode': 'preFilter'}


In [106]:
query_results['value']

[{'@search.score': 0.032522473484277725,
  'parent_storage_path': 'https://stayjdfpgpoyvkc.blob.core.windows.net/text/surface-pro/sample-pdf.pdf',
  'application_name': 'surface-pro',
  'chunk': 'sold \n\nseparately. Only Surface Pro Signature Keyboard has Surface Slim Pen 2 storage and charging capabilities. \n3 Surface Pro 8 with LTE Advanced is coming in 2022. Visit Surface.com for updates on availability in your market. Availability may \n\nvary by market and configuration. Service availability and performance subject to service provider’s network. Contact your service \n\nprovider for details, compatibility, pricing, SIM card, and activation. See all specs and frequencies at surface.com. \n4 Up to 16 hours of battery life based on typical Surface device usage. Testing conducted by Microsoft in August 2021 using \n\npreproduction software and preproduction Intel® 11th Gen Core™ i5-1135G7, 256GB, 8GB RAM device. Testing consisted of full \n\nbattery discharge with a mixture of activ

In [45]:
llm_context = ""
for result in query_results['value']:
    llm_context += f"{result['chunk']} "
logging.info(llm_context)

INFO:root:sold 

separately. Only Surface Pro Signature Keyboard has Surface Slim Pen 2 storage and charging capabilities. 
3 Surface Pro 8 with LTE Advanced is coming in 2022. Visit Surface.com for updates on availability in your market. Availability may 

vary by market and configuration. Service availability and performance subject to service provider’s network. Contact your service 

provider for details, compatibility, pricing, SIM card, and activation. See all specs and frequencies at surface.com. 
4 Up to 16 hours of battery life based on typical Surface device usage. Testing conducted by Microsoft in August 2021 using 

preproduction software and preproduction Intel® 11th Gen Core™ i5-1135G7, 256GB, 8GB RAM device. Testing consisted of full 

battery discharge with a mixture of active use and modern standby. The active use portion consists of (1) a web browsing test 

accessing 8 popular websites over multiple open tabs, (2) a productivity test utilizing Microsoft Word, PowerPo

In [71]:
QUERY = "Microsoft Surface Pro"
text_query_results = query_ai_search(query=QUERY, 
                search_index_name=vector_index_name, 
                text_fields=["chunk"], 
                top=10,  
                select_fields=["parent_storage_path", "application_name", "chunk"], 
                # vector_fields=["embedding"], 
                application_name = app_name_t1)
text_query_results

{'search': 'Microsoft Surface Pro', 'searchFields': 'chunk', 'top': 10, 'filter': "application_name eq 'surface-pro'", 'select': 'parent_storage_path,application_name,chunk'}


{'@odata.context': "https://acsvector-ayjdfpgpoyvkc.search.windows.net/indexes('multi-tenant-index-vector')/$metadata#docs(*)",
 'value': [{'@search.score': 1.7460339,
   'parent_storage_path': 'https://stayjdfpgpoyvkc.blob.core.windows.net/text/surface-pro/sample-pdf.pdf',
   'application_name': 'surface-pro',
   'chunk': '11th Gen Intel® Core™ i5-1135G7 Processor \n\nQuad-core 11th Gen Intel® Core™ i7-1185G7 Processor \n\nDesigned on the Intel® Evo™ platform \n\nCommercial: \n\nDual-core 11th Gen Intel® Core™ i3-1115G4 Processor (Wi-Fi) \n\nQuad-core 11th Gen Intel® Core™ i5-1145G7 Processor (Wi-Fi or LTE3) \n\nQuad-core 11th Gen Intel® Core™ i7-1185G7 Processor (Wi-Fi or LTE3) \n\nDesigned on the Intel® Evo™ platform (i5 and i7 processors only) \n\nGraphics \nIntel® UHD Graphics (i3) \n\nIntel® Iris® Xe Graphics (i5, i7) \n\nMemory 8GB/16GB/32GB LPDDDR4x RAM \n\nStorage \nRemovable solid-state drive (SSD) options: 128GB or 256GB (Wi-Fi or LTE3) \n\nSSD: 512GB or 1TB (Wi-Fi only) \n\

### Step 5 - Use LLM to generate answer

In [47]:
metaprompt = """
    You are a customer service representative for a company that sells Microsoft Surface Pro laptops.
    You have been asked to respond to customer queries for the Microsoft Surface Pro laptop.
    You will only answer from the information provided to you in the search results.
    If the customer asks any other questions, you will politely decline to answer.
"""

In [48]:
def generate_llm_response(query, llm_context, metaprompt, max_tokens=100):
    from openai import AzureOpenAI
        
    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version="2024-02-01",
        azure_endpoint = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
        )
    deployment_name = "gpt-35-turbo"
    
    messages = [
        {
            "role": "system",
            "content": metaprompt
        },
        {
            "role": "user",
            "content": llm_context + "\n\n" + "-"*10 + query
        }
    ]
    
    response = client.chat.completions.create(model=deployment_name, 
                                         messages=messages, 
                                         max_tokens=max_tokens)
    
    return response.choices[0].message.content

In [57]:
user_input = "What is the battery life of the Microsoft Surface Pro?"
result = generate_llm_response(query=user_input, llm_context=llm_context, metaprompt=metaprompt, max_tokens=100)
pprint.pprint(result)

INFO:httpx:HTTP Request: POST https://cog-ayjdfpgpoyvkc.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


('According to the information provided, the Microsoft Surface Pro 8 has up to '
 '16 hours of battery life based on typical Surface device usage. However, '
 'battery life can vary significantly with settings, usage, and other factors.')


-------------